In [1]:
# Загрузка fastaparser для анализа файлов расширения .fa
%pip install fastaparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Импорт необходимых для работы пакетов
import fastaparser as fst
import pandas as pd
import re

Перечень функций для анализа:

In [3]:
# Создание pandas.DataFrame на основе файла .fa
def df_parser(file_path):
    with open(file_path, "r") as file:
        # Используем быстрый способ парсинга, он позволяет получить всю необходимую информацию быстрее.
        reader = fst.Reader(file, parse_method = 'quick')
        df = pd.DataFrame(columns = ['Sequence', 'Length'])
        # Перебираем объекты FastaSequence и формируем DataFrame.
        for seq in reader:
            df.loc[seq.header, 'Sequence'] = seq.sequence  # Сама последовательность
            df.loc[seq.header, 'Length'] = len(seq.sequence)  # Длина последовательности
    # Используем сортировку ПО УБЫВАНИЮ для дальнейшего рассчета 50N.
    # Также самый длинный скаффолд окажется первым элементом фрейма.
    return df.sort_values(by = 'Length', ascending = False)

# Получение результата на основе анализа полученного DataFrame.
def analysis(df):
    print("Общее количество: ", len(df))
    print("Общая длина: ", df.Length.sum())
    print("Самая большая длина: ", df.Length[0])
    # Расчет 50N
    index = 1
    amount = df.Length[0]
    # Пока текущая сумма не менее половины общей длины всех контигов сборки.
    while (amount < df.Length.sum() / 2):
        amount += df.Length[index]
        index += 1
    print("N50: ", df.Length[index - 1])

# Подсчет гэпов в последовательности. 
def gaps_count(sequence):
    # Используем регулярное выражение для поисков участков из N.
    print("Количество гэпов: ", len(re.findall("N+", sequence)))
    print("Общая длина гэпов: ", sequence.count("N"))

Произведение расчетов.
Код использовался для двух геномов поочередно, чтобы не загромождать ноутбук.

In [6]:
# Основная программа.
contigs = df_parser('Poil_contig.fa')
scaffolds = df_parser('Poil_scaffold.fa')
gapClosed = df_parser('Poil_gapClosed.fa')

print("Анализ контигов:")
analysis(contigs)
print("\nАнализ скаффолдов:")
analysis(scaffolds)
print('\nАнализ гэпов для самого длинного скаффолда:')
gaps_count(scaffolds.Sequence[0])

print('\n---ПОСЛЕ УМЕНЬШЕНИЯ КОЛИЧЕСТВА ГЭПОВ---\n')
print("Анализ скаффолдов:")
analysis(gapClosed)
print('\nАнализ гэпов самого длинного скаффолда:')
gaps_count(gapClosed.Sequence[0])

Анализ контигов:
Общее количество:  713
Общая длина:  3924600
Самая большая длина:  235702
N50:  75466

Анализ скаффолдов:
Общее количество:  78
Общая длина:  3867546
Самая большая длина:  3833962
N50:  3833962

Анализ гэпов для самого длинного скаффолда:
Количество гэпов:  74
Общая длина гэпов:  7988

---ПОСЛЕ УМЕНЬШЕНИЯ КОЛИЧЕСТВА ГЭПОВ---

Анализ скаффолдов:
Общее количество:  78
Общая длина:  3893717
Самая большая длина:  3860126
N50:  3860126

Анализ гэпов самого длинного скаффолда:
Количество гэпов:  22
Общая длина гэпов:  4450


In [ ]:
# Запись самого длинного скаффолда в файл.
def write_scaffold(file_path):
  with open(file_path, 'w') as file:
    writer = fst.Writer(file)
    writer.writefasta((gapClosed.index[0], gapClosed.Sequence[0]))

write_scaffold('longest.fasta')